In [1]:
import requests
import pandas as pd

# Example location: Chennai, India
lat, lon = 13.0827, 80.2707
params = {
    "parameters": "T2M_MAX,T2M_MIN,PRECTOTCORR,ALLSKY_SFC_SW_DWN,RH2M,WS2M,EVPTRNS",
    "community": "AG",
    "longitude": lon,
    "latitude": lat,
    "start": "20250101",
    "end": "20250110",
    "format": "JSON"
}

url = "https://power.larc.nasa.gov/api/temporal/daily/point"
response = requests.get(url, params=params).json()

# Convert to DataFrame
data = response['properties']['parameter']
df = pd.DataFrame(data)
print(df.head())


          T2M_MAX  T2M_MIN  PRECTOTCORR  ALLSKY_SFC_SW_DWN   RH2M  WS2M  \
20250101    27.49    22.28         0.00              15.84  83.31  3.27   
20250102    27.22    21.54         0.03              15.74  82.63  3.01   
20250103    27.07    21.43         0.01              17.03  79.01  3.09   
20250104    26.76    20.62         0.02              17.82  77.78  2.47   
20250105    27.30    20.81         0.00              16.21  76.19  2.03   

          EVPTRNS  
20250101     1.86  
20250102     1.89  
20250103     1.94  
20250104     2.00  
20250105     2.13  


In [2]:
def fetch_weather(lat, lon, start, end, site_name="SITE", inst="NASA", insi="NA"):
    import requests, pandas as pd

    url = f"https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "community": "AG",
        "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2M,RH2M,PRECTOTCORR,WS2M",
        "format": "json",
    }
    r = requests.get(url, params=params)
    response = r.json()

    data = response["properties"]["parameter"]

    # Build with dates as rows (orient='index')
    df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    # Rename columns for DSSAT
    df = df.rename(
        columns={
            "ALLSKY_SFC_SW_DWN": "srad",
            "T2M_MAX": "tmax",
            "T2M_MIN": "tmin",
            "T2M": "tavg",
            "PRECTOTCORR": "rain",
            "WS2M": "wind",
            "RH2M": "rhum",
        }
    )

    return df
df = fetch_weather(-29.6, 30.0, "20250101", "20250107")
print(df.head())



        date   srad   tmax   tmin   tavg   rhum   rain  wind
0 2025-01-01  11.09  25.04  16.18  19.96  81.31   5.70  2.17
1 2025-01-02  17.72  26.89  16.56  21.17  75.43   2.56  2.18
2 2025-01-03  29.52  32.58  17.43  24.58  66.77   0.51  2.22
3 2025-01-04  30.11  35.41  19.23  26.80  58.39   0.94  1.89
4 2025-01-05  16.36  27.22  18.41  22.42  77.67  32.73  1.19


In [3]:
import requests
import pandas as pd

def fetch_weather(lat, lon, start, end, site_name="SITE", inst="NASA", insi="XXXX"):
    """
    Fetch daily weather data from NASA POWER and return DSSAT-style DataFrame.

    Parameters
    ----------
    lat : float
        Latitude
    lon : float
        Longitude
    start : str
        Start date (YYYYMMDD)
    end : str
        End date (YYYYMMDD)
    site_name : str
        Site name
    inst : str
        Institution name (unused here, kept for WTH file format)
    insi : str
        INSI code (4-character station code, e.g., 'SATO')
    """

    # --- NASA POWER API ---
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "community": "AG",
        "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2M,RH2M,PRECTOTCORR,WS2M",
        "format": "json",
    }
    r = requests.get(url, params=params)
    response = r.json()

    # --- Build DataFrame (dates as rows) ---
    data = response["properties"]["parameter"]
    df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    # --- Rename columns for DSSAT ---
    df = df.rename(
        columns={
            "ALLSKY_SFC_SW_DWN": "srad",  # MJ m-2 day-1
            "T2M_MAX": "tmax",            # °C
            "T2M_MIN": "tmin",            # °C
            "T2M": "tavg",                # °C
            "PRECTOTCORR": "rain",        # mm
            "WS2M": "wind",               # m/s (can scale if needed)
            "RH2M": "rhum",               # %
        }
    )

    # --- Derived column: YRDOY (2-digit year + DOY) ---
    df["YRDOY"] = df["date"].dt.strftime("%y%j")

    # --- Add metadata ---
    df["INSI"] = insi
    df["SITE"] = site_name
    df["LAT"] = lat
    df["LON"] = lon

    return df


# ----------------- Example Usage -----------------
if __name__ == "__main__":
    # Example: fetch data for India (Chennai, approx)
    df = fetch_weather(13.08, 80.27, "20250101", "20250107", site_name="CHENNAI", insi="IN01")

    print(df.head())

    # Save as CSV (can later format into WTH file)
    df.to_csv("weather_data.csv", index=False)


        date   srad   tmax   tmin   tavg   rhum  rain  wind  YRDOY  INSI  \
0 2025-01-01  15.84  27.49  22.28  24.48  83.31  0.00  3.27  25001  IN01   
1 2025-01-02  15.74  27.22  21.54  24.06  82.63  0.03  3.01  25002  IN01   
2 2025-01-03  17.03  27.07  21.43  23.63  79.01  0.01  3.09  25003  IN01   
3 2025-01-04  17.82  26.76  20.62  23.48  77.78  0.02  2.47  25004  IN01   
4 2025-01-05  16.21  27.30  20.81  23.74  76.19  0.00  2.03  25005  IN01   

      SITE    LAT    LON  
0  CHENNAI  13.08  80.27  
1  CHENNAI  13.08  80.27  
2  CHENNAI  13.08  80.27  
3  CHENNAI  13.08  80.27  
4  CHENNAI  13.08  80.27  


In [4]:
import requests
import pandas as pd
import numpy as np

def fetch_weather(lat, lon, start, end, site_name="SITE", inst="NASA", insi="XXXX"):
    """
    Fetch daily weather data from NASA POWER and save as DSSAT-style .WTH file.
    """

    # --- NASA POWER API ---
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start,
        "end": end,
        "latitude": lat,
        "longitude": lon,
        "community": "AG",
        "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2M,RH2M,PRECTOTCORR,WS2M",
        "format": "json",
    }
    r = requests.get(url, params=params)
    response = r.json()

    # --- Build DataFrame ---
    data = response["properties"]["parameter"]
    df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    # --- Rename ---
    df = df.rename(
        columns={
            "ALLSKY_SFC_SW_DWN": "srad",  # MJ m-2 day-1
            "T2M_MAX": "tmax",
            "T2M_MIN": "tmin",
            "T2M": "tavg",
            "PRECTOTCORR": "rain",
            "WS2M": "wind",               # m/s
            "RH2M": "rhum",
        }
    )

    # --- Derived ---
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    df["wind"] = df["wind"] * 86.4   # convert m/s → km/day (DSSAT expects km/day)

    # --- TAV (mean temp) and AMP (annual range) ---
    tav = df["tavg"].mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    # --- Prepare columns for DSSAT ---
    df_out = df[["YRDOY", "srad", "tmax", "tmin", "rain", "wind", "rhum"]]

    # --- Write .WTH file ---
    filename = f"{insi}{start[2:4]}.WTH"  # e.g., IN011.WTH for year 2025
    with open(filename, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},{inst}\n\n")
        f.write("@ INSI      LAT     LONG  ELEV   TAV   AMP  REFHT  WNDHT\n")
        f.write(f"  {insi:<4s}   {lat:7.3f} {lon:7.3f}   100  {tav:5.1f}  {amp:5.1f}   2.0   2.0\n\n")

        f.write("@DATE  SRAD  TMAX  TMIN  RAIN  WIND  RHUM\n")
        for _, row in df_out.iterrows():
            f.write(f"{int(row.YRDOY):5d}{row.srad:6.1f}{row.tmax:6.1f}{row.tmin:6.1f}"
                    f"{row.rain:6.1f}{row.wind:6.1f}{row.rhum:6.1f}\n")

    print(f"✅ WTH file saved as: {filename}")
    return df_out


# ----------------- Example -----------------
if __name__ == "__main__":
    df = fetch_weather(13.08, 80.27, "20250101", "20250107", site_name="CHENNAI", insi="IN01")
    print(df.head())


KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import numpy as np

def fetch_weather_with_extras(lat, lon, start, end,
                              site_name="SITE", inst="NASA", insi="XXXX",
                              elev=100.0):
    """
    Fetch daily weather from NASA POWER and write DSSAT .WTH including DEWP, VAPR, PAR, EVAP.
    start/end in YYYYMMDD
    """
    # Request parameters (include T2MDEW and EVPTRNS)
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start, "end": end,
        "latitude": lat, "longitude": lon,
        "community": "AG",
        "parameters": ",".join([
            "ALLSKY_SFC_SW_DWN",  # srad (MJ m-2 day-1)
            "T2M_MAX", "T2M_MIN", "T2M",  # temps
            "PRECTOTCORR",         # rain (mm/day)
            "T2MDEW",              # dewpoint (°C)
            "WS2M",                # wind m/s
            "EVPTRNS",             # evapotranspiration (mm/day)
            "RH2M"                 # rel humidity (%)
        ]),
        "format": "JSON"
    }
    r = requests.get(url, params=params)
    r.raise_for_status()
    resp = r.json()

    # Build DataFrame with dates as rows
    data = resp["properties"]["parameter"]
    df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    # Rename to friendly columns
    df = df.rename(columns={
        "ALLSKY_SFC_SW_DWN": "srad",
        "T2M_MAX": "tmax",
        "T2M_MIN": "tmin",
        "T2M": "tavg",
        "PRECTOTCORR": "rain",
        "T2MDEW": "dewp",
        "WS2M": "wind",   # m/s
        "EVPTRNS": "evap",
        "RH2M": "rhum"
    })

    # Convert wind m/s -> km/day (DSSAT often used km/day)
    df["wind"] = df["wind"] * 86.4

    # YRDOY (2-digit year + 3-digit DOY)
    df["YRDOY"] = df["date"].dt.strftime("%y%j")

    # --- Compute VAPR (kPa) from DEWP (°C) ---
    # saturated vapor pressure formula at dewpoint
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))

    # --- Compute PAR as monthly average of SRAD then convert to mol m-2 day-1 ---
    # 1) monthly mean SRAD (MJ/m2/day)
    df["month"] = df["date"].dt.month
    srad_monthly = df.groupby("month")["srad"].transform("mean")  # same length as df
    # 2) Convert monthly SRAD -> PAR using conversion factor (MJ -> mol)
    #    PAR (mol m-2 d-1) ≈ SRAD (MJ/m2/day) * (2.04 / 0.218)
    conv = 2.04 / 0.218
    df["par"] = srad_monthly * conv

    # --- Placeholders for ozone and CO2 (could be fetched from other APIs) ---
    df["ozon7"] = 50        # ppb (placeholder)
    df["dco2"] = 421.0      # ppm (placeholder)

    # --- Header stats: TAV and AMP ---
    df["tavg"] = (df["tmax"] + df["tmin"]) / 2.0
    tav = df["tavg"].mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    # --- Prepare WTH filename and header (extract elevation if available) ---
    # Try to extract elevation from NASA header if present, fallback to provided elev
    header_meta = resp.get("header", {}) or {}
    elev_api = header_meta.get("elevation")
    elev_to_use = elev_api if elev_api is not None else elev

    filename = f"{insi}{start[2:4]}.WTH"  # e.g., IN011.WTH

    # --- Write .WTH with extra columns ---
    with open(filename, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},{inst}\n\n")
        f.write("@ INSI      LAT     LONG  ELEV   TAV   AMP  REFHT  WNDHT  CCO2\n")
        f.write(f"  {insi:<4s}   {lat:7.3f} {lon:7.3f}   {elev_to_use:5.0f}  {tav:5.1f}  {amp:5.1f}   2.0   2.0  {df['dco2'].iloc[0]:6.1f}\n\n")

        # Column header includes the extras (DEWP, TDEW==DEWP, PAR, VAPR, VPRS==VAPR, DCO2, OZON7, EVAP)
        f.write("@  DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP  TDEW   PAR   VAPR  VPRS   DCO2  OZON7  EVAP\n")
        for _, row in df.iterrows():
            # DATE should be numeric YRDOY like 25001
            date_num = int(row["YRDOY"])
            f.write(
                f"{date_num:7d} "
                f"{row['srad']:5.1f} "
                f"{row['tmax']:5.1f} "
                f"{row['tmin']:5.1f} "
                f"{row['rain']:6.1f} "
                f"{row['rhum']:5.1f} "
                f"{row['wind']:6.1f} "
                f"{row['dewp']:5.1f} "    # DEWP
                f"{row['dewp']:6.1f} "    # TDEW (DSSAT sometimes uses TDEW)
                f"{row['par']:6.1f} "
                f"{row['vapr']:6.2f} "
                f"{row['vapr']:6.2f} "
                f"{row['dco2']:6.1f} "
                f"{int(row['ozon7']):6d} "
                f"{row['evap']:6.2f}\n"
            )

    print(f"Saved WTH: {filename}")
    return df, filename

# ---------------- Example usage ----------------
if __name__ == "__main__":
    df, fn = fetch_weather_with_extras(
        lat=13.08, lon=80.27,
        start="20250101", end="20250107",
        site_name="CHENNAI", inst="NASA", insi="IN01",
        elev=6
    )
    print(df.head())


Saved WTH: IN0125.WTH
        date   srad   tmax   tmin    tavg  rain   dewp     wind  evap   rhum  \
0 2025-01-01  15.84  27.49  22.28  24.885  0.00  21.41  282.528  1.86  83.31   
1 2025-01-02  15.74  27.22  21.54  24.380  0.03  20.85  260.064  1.89  82.63   
2 2025-01-03  17.03  27.07  21.43  24.250  0.01  19.67  266.976  1.94  79.01   
3 2025-01-04  17.82  26.76  20.62  23.690  0.02  19.29  213.408  2.00  77.78   
4 2025-01-05  16.21  27.30  20.81  24.055  0.00  19.21  175.392  2.13  76.19   

   YRDOY      vapr  month         par  ozon7   dco2  
0  25001  2.550332      1  157.317955     50  421.0  
1  25002  2.464183      1  157.317955     50  421.0  
2  25003  2.290939      1  157.317955     50  421.0  
3  25004  2.237465      1  157.317955     50  421.0  
4  25005  2.226347      1  157.317955     50  421.0  


In [42]:
import requests
import pandas as pd
import numpy as np
import os

def fetch_weather(lat, lon, start, end, site_name="SITE", inst="NASA", insi="XXXX", elev=None):
    # Convert dates from yyyy-mm-dd → YYYYMMDD for NASA API
    start_fmt = pd.to_datetime(start).strftime("%Y%m%d")
    end_fmt   = pd.to_datetime(end).strftime("%Y%m%d")

    # Create output directory
    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)

    # NASA POWER API
    url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    params = {
        "start": start_fmt,
        "end": end_fmt,
        "latitude": lat,
        "longitude": lon,
        "community": "AG",
        "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
        "format": "JSON"
    }
    r = requests.get(url, params=params)
    response = r.json()
    data = response["properties"]["parameter"]

    # Only keep date keys (YYYYMMDD) for the first parameter
    date_keys = [k for k in data[list(data.keys())[0]].keys() if k.isdigit()]

    # Build DataFrame
    df = pd.DataFrame({param: {date: data[param][date] for date in date_keys} for param in data})
    df = df.reset_index().rename(columns={"index":"date"})
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")

    # Rename columns for DSSAT
    df = df.rename(columns={
        "ALLSKY_SFC_SW_DWN":"srad",
        "T2M_MAX":"tmax",
        "T2M_MIN":"tmin",
        "T2MDEW":"dewp",
        "PRECTOTCORR":"rain",
        "WS2M":"wind",
        "RH2M":"rhum",
        "EVPTRNS":"evap"
    })

    # Unit conversions
    df["wind"] = df["wind"] * 86.4  # m/s → km/day

    # Monthly avg SRAD → PAR
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)

    # Vapor pressure: Calculated from the dew point temperature using the Magnus-Tetens formula.
    df["vapr"] = 0.6108 * np.exp((17.27*df["dewp"])/(df["dewp"]+237.3)) 

    df["ozon7"] = 50
    df["dco2"] = 421.0

    # DSSAT YRDOY
    df["YRDOY"] = df["date"].dt.strftime("%y%j")

    # TAV and AMP
    tav = ((df["tmax"] + df["tmin"])/2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    # Prepare output
    df_out = df[["YRDOY","srad","tmax","tmin","rain","rhum","wind",
                 "dewp","par","vapr","dco2","ozon7","evap"]]

    # Save WTH file
    start_yrdoy = df_out["YRDOY"].iloc[0]
    wth_file = os.path.join(output_dir, f"{insi}_{start_yrdoy}.WTH")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},{inst}\n\n")
        f.write("@ INSI      LAT     LONG  ELEV   TAV   AMP  REFHT  WNDHT  CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else 100:5.1f}  {tav:5.1f}  {amp:5.1f}  2.0  2.0  {df['dco2'].iloc[0]:6.1f}\n\n")
        f.write("@  DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP  PAR  VAPR  DCO2  OZON7  EVAP\n")
        for _, row in df_out.iterrows():
            f.write(f"{int(row.YRDOY):7d} "
                    f"{row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
                    f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} "
                    f"{row.dewp:5.1f} {row.par:6.1f} {row.vapr:5.2f} "
                    f"{row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")

    print(f"WTH saved at: {wth_file}")
    return df_out

# ----------------- Example Usage -----------------
if __name__ == "__main__":
    df = fetch_weather(13.08, 80.27, "2025-01-01", "2025-03-31", site_name="CHENNAI", insi="IN01", elev=6)
    print(df.head())

WTH saved at: weather\IN01_25001.WTH
   YRDOY   srad   tmax   tmin  rain   rhum     wind   dewp        par  \
0  25001  15.84  27.49  22.28  0.00  83.31  282.528  21.41  16.415161   
1  25002  15.74  27.22  21.54  0.03  82.63  260.064  20.85  16.415161   
2  25003  17.03  27.07  21.43  0.01  79.01  266.976  19.67  16.415161   
3  25004  17.82  26.76  20.62  0.02  77.78  213.408  19.29  16.415161   
4  25005  16.21  27.30  20.81  0.00  76.19  175.392  19.21  16.415161   

       vapr   dco2  ozon7  evap  
0  2.550332  421.0     50  1.86  
1  2.464183  421.0     50  1.89  
2  2.290939  421.0     50  1.94  
3  2.237465  421.0     50  2.00  
4  2.226347  421.0     50  2.13  


In [8]:
API_KEY = "eddbc7c0c9e63e225934add809624c6e"

In [12]:
# only ozone and co2 retrival
import requests
import json
from datetime import datetime, timezone, timedelta

# --- Configuration ---
# ❗️ PASTE YOUR API KEY HERE
# API_KEY = "YOUR_API_KEY_HERE"

# Specify the coordinates for your location
# LATITUDE = 28.6139  # Delhi
# LONGITUDE = 77.2090
LATITUDE = 13.08  # chennai
LONGITUDE = 80.27

# Specify the date range (Format: YYYY-MM-DD)
# The script will fetch all hourly data from the start of the START_DATE
# to the very end of the END_DATE.
START_DATE = "2025-09-01"
END_DATE = "2025-09-03"

# --- Main Script ---

def get_historical_air_quality(api_key, lat, lon, start_str, end_str):
    """
    Fetches, processes, and displays historical air quality data in a clean table.
    """
    if api_key == "YOUR_API_KEY_HERE":
        print("🛑 Error: Please replace 'YOUR_API_KEY_HERE' with your actual OpenWeatherMap API key.")
        return

    # A dictionary to map the AQI index to its qualitative name
    aqi_map = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}
    
    # Define the local timezone for India (IST is UTC+5:30)
    ist_tz = timezone(timedelta(hours=5, minutes=30))

    try:
        # Step 1: Convert human-readable dates to UNIX timestamps.
        # We set the end date to the last second of the day to ensure the entire day is included.
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        
        start_timestamp = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_timestamp = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        # Step 2: Build the API URL for historical data.
        api_url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
                   f"lat={lat}&lon={lon}&start={start_timestamp}&end={end_timestamp}&appid={api_key}")

        # Step 3: Make the API request and handle potential errors.
        response = requests.get(api_url)
        response.raise_for_status()  # This will raise an exception for HTTP error codes (4xx or 5xx)
        data = response.json()

        # Step 4: Process and display the data in a formatted table.
        print("----------------------------------------------------------------------------------------------------------------")
        print(f"📍 Historical Air Quality for Delhi ({lat}, {lon}) from {start_str} to {end_str}")
        print("----------------------------------------------------------------------------------------------------------------")
        
        # Print table header
        header = (f"{'Date & Time (IST)':<22} | {'AQI':<12} | {'PM2.5':>7} | {'PM10':>7} | {'O₃':>7} | "
                  f"{'NO₂':>7} | {'SO₂':>7} | {'CO':>9}")
        print(header)
        print("-" * len(header))

        if 'list' in data and data['list']:
            for entry in data['list']:
                # Convert timestamp to local IST datetime object
                local_time = datetime.fromtimestamp(entry['dt'], tz=ist_tz)
                
                # Get AQI and its text description
                aqi_index = entry['main']['aqi']
                aqi_text = aqi_map.get(aqi_index, "Unknown")
                
                # Get all component values, using .get() for safety
                components = entry['components']
                pm2_5 = components.get('pm2_5', 0)
                pm10 = components.get('pm10', 0)
                o3 = components.get('o3', 0)
                no2 = components.get('no2', 0)
                so2 = components.get('so2', 0)
                co = components.get('co', 0)
                
                # Print the formatted row for the table
                print(f"{local_time.strftime('%Y-%m-%d %H:%M:%S'):<22} | {aqi_text:<12} | "
                      f"{pm2_5:7.2f} | {pm10:7.2f} | {o3:7.2f} | "
                      f"{no2:7.2f} | {so2:7.2f} | {co:9.2f}")
        else:
            print("No data found for the specified location and date range.")

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print(f"HTTP Error 401: Unauthorized. Please check if your API key is correct and active.")
        else:
            print(f"An HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"A network error occurred: {e}")
    except (KeyError, IndexError) as e:
        print(f"Error parsing data: {e}. The response from the API might not be in the expected format.")
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")

# --- Run the script ---
if __name__ == "__main__":
    get_historical_air_quality(API_KEY, LATITUDE, LONGITUDE, START_DATE, END_DATE)

----------------------------------------------------------------------------------------------------------------
📍 Historical Air Quality for Delhi (13.08, 80.27) from 2025-09-01 to 2025-09-03
----------------------------------------------------------------------------------------------------------------
Date & Time (IST)      | AQI          |   PM2.5 |    PM10 |      O₃ |     NO₂ |     SO₂ |        CO
---------------------------------------------------------------------------------------------------
2025-09-01 05:30:00    | Good         |    9.46 |   12.90 |   11.83 |   11.52 |    3.11 |    229.68
2025-09-01 06:30:00    | Good         |    7.43 |   10.48 |   13.74 |    9.49 |    2.79 |    202.35
2025-09-01 07:30:00    | Good         |    6.45 |    9.83 |   14.52 |    8.59 |    2.85 |    196.42
2025-09-01 08:30:00    | Good         |    6.10 |    9.98 |   15.54 |    8.08 |    3.81 |    198.79
2025-09-01 09:30:00    | Good         |    5.49 |    9.00 |   17.74 |    7.93 |    9.54 |    1

In [16]:
import requests
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import statistics

# --- Configuration ---
# ❗️ PASTE YOUR API KEY HERE
# API_KEY = "YOUR_API_KEY_HERE"

# Specify the coordinates for your location
# LATITUDE = 28.6139  # Delhi
# LONGITUDE = 77.2090
LATITUDE = 11.202263   # chennai
LONGITUDE = 76.96184
# LATITUDE = 13.08  # chennai
# LONGITUDE = 80.27

# Specify the date range (Format: YYYY-MM-DD)
START_DATE = "2025-09-01"
END_DATE = "2025-10-31"

# --- Main Script ---

def get_daily_average_air_quality(api_key, lat, lon, start_str, end_str):
    """
    Fetches historical data, calculates daily averages, and displays a summary table.
    """
    if api_key == "YOUR_API_KEY_HERE":
        print("🛑 Error: Please replace 'YOUR_API_KEY_HERE' with your actual OpenWeatherMap API key.")
        return

    aqi_map = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}
    ist_tz = timezone(timedelta(hours=5, minutes=30))

    try:
        # Step 1: Convert dates to UNIX timestamps
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        start_timestamp = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_timestamp = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        # Step 2: Build the API URL and make the request
        api_url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
                   f"lat={lat}&lon={lon}&start={start_timestamp}&end={end_timestamp}&appid={api_key}")
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        # Step 3: Aggregate hourly data by date
        # We use a defaultdict to easily create lists for each new date.
        daily_data = defaultdict(lambda: defaultdict(list))

        if 'list' in data and data['list']:
            for entry in data['list']:
                # Get the date in the local IST timezone
                local_date = datetime.fromtimestamp(entry['dt'], tz=ist_tz).date()
                
                # Add the hourly data to the list for that specific date
                daily_data[local_date]['aqi'].append(entry['main']['aqi'])
                for component, value in entry['components'].items():
                    daily_data[local_date][component].append(value)
        else:
            print("No data found for the specified location and date range.")
            return

        # Step 4: Calculate averages and display the summary table
        print("-------------------------------------------------------------------------------------------------")
        print(f"📊 Daily Average Air Quality for Delhi ({lat}, {lon})")
        print("-------------------------------------------------------------------------------------------------")
        
        header = (f"{'Date':<12} | {'Dominant AQI':<15} | {'PM2.5':>7} | {'PM10':>7} | {'O₃':>7} | "
                  f"{'NO₂':>7} | {'SO₂':>7} | {'CO':>9}")
        print(header)
        print("-" * len(header))

        # Sort the dates before printing
        for date in sorted(daily_data.keys()):
            day = daily_data[date]
            
            # Calculate the average for each pollutant
            # The inner defaultdict ensures we don't get an error for missing components
            avg_pm2_5 = sum(day['pm2_5']) / len(day['pm2_5'])
            avg_pm10 = sum(day['pm10']) / len(day['pm10'])
            avg_o3 = sum(day['o3']) / len(day['o3'])
            avg_no2 = sum(day['no2']) / len(day['no2'])
            avg_so2 = sum(day['so2']) / len(day['so2'])
            avg_co = sum(day['co']) / len(day['co'])
            
            # Find the most frequent (mode) AQI for the day
            try:
                modal_aqi_index = statistics.mode(day['aqi'])
            except statistics.StatisticsError: # Handles cases with no unique mode
                modal_aqi_index = day['aqi'][0] # Fallback to the first value
            
            modal_aqi_text = aqi_map.get(modal_aqi_index, "Unknown")
            
            # Print the formatted row of daily averages
            print(f"{date.strftime('%Y-%m-%d'):<12} | {modal_aqi_text:<15} | "
                  f"{avg_pm2_5:7.2f} | {avg_pm10:7.2f} | {avg_o3:7.2f} | "
                  f"{avg_no2:7.2f} | {avg_so2:7.2f} | {avg_co:9.2f}")

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print(f"HTTP Error 401: Unauthorized. Please check if your API key is correct and active.")
        else:
            print(f"An HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"A network error occurred: {e}")
    except (KeyError, IndexError, ZeroDivisionError) as e:
        print(f"Error processing data: {e}. The API response may be incomplete.")
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")


# --- Run the script ---
if __name__ == "__main__":
    get_daily_average_air_quality(API_KEY, LATITUDE, LONGITUDE, START_DATE, END_DATE)

-------------------------------------------------------------------------------------------------
📊 Daily Average Air Quality for Delhi (11.202263, 76.96184)
-------------------------------------------------------------------------------------------------
Date         | Dominant AQI    |   PM2.5 |    PM10 |      O₃ |     NO₂ |     SO₂ |        CO
--------------------------------------------------------------------------------------------
2025-09-01   | Good            |    3.12 |    4.93 |   24.46 |    3.71 |    1.59 |    124.25
2025-09-02   | Good            |    3.69 |    7.40 |   27.94 |    3.51 |    1.68 |    123.11
2025-09-03   | Good            |    3.88 |    6.89 |   26.71 |    4.34 |    1.76 |    136.83
2025-09-04   | Good            |    3.75 |    6.28 |   26.20 |    4.97 |    1.84 |    149.39
2025-09-05   | Good            |    4.20 |    6.78 |   29.92 |    4.87 |    1.92 |    146.98
2025-09-06   | Good            |    4.20 |    6.84 |   29.28 |    3.53 |    1.64 |    124.26


In [22]:
import requests

# The URL of the data file
url = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.csv'

# The name of the output file
output_filename = 'co2_daily_mlo.txt'

try:
    # 1. Send an HTTP GET request to the URL
    print(f"Fetching data from {url}...")
    response = requests.get(url, timeout=10)

    # Raise an exception for bad status codes (4xx or 5xx)
    response.raise_for_status()

    # 2. Save the content to a text file
    print(f"Saving data to {output_filename}...")
    with open(output_filename, 'w', encoding='utf-8') as file:
        file.write(response.text)

    print("✅ Data fetching and saving complete!")

except requests.exceptions.RequestException as e:
    print(f"Error: Could not fetch the URL. {e}")
except IOError as e:
    print(f"Error: Could not write to the file. {e}")

Fetching data from https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.csv...
Saving data to co2_daily_mlo.txt...
✅ Data fetching and saving complete!


In [ ]:
import pandas as pd

# Correctly read the space-delimited file
df = pd.read_csv(
    "./co2_daily_mlo.csv",
    comment="#",
    names=["yr", "mon", "day", "decimal", "co2"]
)

print("The last row is:")

# Use .to_string(index=False) to print the DataFrame without its index
print(df.tail(1).to_string(index=False))

The last row is:
  yr  mon  day   decimal   ppm
2025    9   12 2025.6959 424.4


In [61]:
import requests
import pandas as pd
import os
from datetime import datetime

def get_co2_data(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
    """
    Fetches daily CO2 data for a given date range, caching the data locally.

    It checks if the data file exists and if the requested date range is
    covered. It only downloads a fresh copy if the file is missing or the
    date range is not fully available.

    Args:
        start_date_str (str): The start date in 'YYYY-MM-DD' format.
        end_date_str (str): The end date in 'YYYY-MM-DD' format.
        filename (str): The local filename to use for caching.

    Returns:
        pd.DataFrame: A DataFrame containing the CO2 data for the specified range,
                      or an empty DataFrame if data cannot be retrieved.
    """
    DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
    
    # Convert input strings to datetime objects for comparison
    req_start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    req_end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    download_needed = False
    
    # 1. Check if the file exists and if the date range is covered
    if os.path.exists(filename):
        print(f"📄 Local file '{filename}' found.")
        try:
            # Read only essential columns to check date range quickly
            local_df_check = pd.read_csv(
                filename,
                comment="#",
                sep=r'\s+',
                usecols=[0, 1, 2],
                names=["yr", "mon", "day"]
            )
            # Rename columns for pd.to_datetime compatibility
            date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
            local_df_check['date'] = pd.to_datetime(local_df_check[['yr', 'mon', 'day']].rename(columns=date_cols))
            
            local_min_date = local_df_check['date'].min()
            local_max_date = local_df_check['date'].max()
            print(f"   - Local data covers: {local_min_date.date()} to {local_max_date.date()}")

            # 2. Decide if a new download is necessary
            if req_start_date < local_min_date or req_end_date > local_max_date:
                print("   - Requested date range is outside local data. A new download is required.")
                download_needed = True
            else:
                print("   - Local data is sufficient for the requested range.")
                
        except (pd.errors.EmptyDataError, KeyError, IndexError):
            print("   - Local file seems corrupt or empty. Re-downloading.")
            download_needed = True
    else:
        print(f"📄 Local file '{filename}' not found. Download is required.")
        download_needed = True

    # 3. Download if needed
    if download_needed:
        try:
            print(f"⬇️ Downloading latest data from {DATA_URL}...")
            response = requests.get(DATA_URL, timeout=15)
            response.raise_for_status()

            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"✅ Download complete. Data saved to {filename}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Error: Could not fetch the URL. {e}")
            return pd.DataFrame()

    # 4. Read the definitive local file and filter it
    print(f"Reading and processing {filename}...")
    try:
        df = pd.read_csv(
            filename,
            comment="#",
            sep=r'\s+',
            names=["yr", "mon", "day", "decimal", "ppm"]
        )
        # Rename columns for pd.to_datetime compatibility
        date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
        df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns=date_cols))
        
        mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
        filtered_df = df.loc[mask].reset_index(drop=True)
        print("✅ Processing complete.")
        return filtered_df[['date', 'ppm']]

    except FileNotFoundError:
        print(f"❌ Error: Could not find {filename} after attempting to download.")
        return pd.DataFrame()

# --- Example Usage ---
if __name__ == "__main__":
    print("--- Request 1: A historical date range (should use local file if available) ---")
    co2_data_2022 = get_co2_data(start_date_str='2022-05-01', end_date_str='2023-04-10')
    if not co2_data_2022.empty:
        print("\nCO2 Data for early May 2022:")
        print(co2_data_2022)

    print("\n" + "="*50 + "\n")


--- Request 1: A historical date range (should use local file if available) ---
📄 Local file 'co2_daily_mlo.txt' found.
   - Local data covers: 1974-05-19 to 2025-09-12
   - Local data is sufficient for the requested range.
Reading and processing co2_daily_mlo.txt...
✅ Processing complete.

CO2 Data for early May 2022:
          date     ppm
0   2022-05-01  419.75
1   2022-05-02  420.94
2   2022-05-03  420.43
3   2022-05-04  421.28
4   2022-05-05  419.63
..         ...     ...
308 2023-04-05  422.75
309 2023-04-06  423.01
310 2023-04-07  422.41
311 2023-04-09  421.72
312 2023-04-10  422.29

[313 rows x 2 columns]




In [80]:
# import requests
# import pandas as pd
# import numpy as np
# import os
# from datetime import datetime, timezone, timedelta
# from collections import defaultdict

# # ==============================================================================
# # --- ⚙️ CONFIGURATION - PLEASE EDIT THESE VALUES ---
# # ==============================================================================

# # ❗️ REQUIRED: Paste your OpenWeatherMap API key here to fetch Ozone data
# OWM_API_KEY = "eddbc7c0c9e63e225934add809624c6e "

# # --- Location Settings ---
# LATITUDE = 13.08    # Example: Chennai
# LONGITUDE = 80.27   # Example: Chennai
# ELEVATION = 6       # Elevation in meters

# # --- Date Range (Format: YYYY-MM-DD) ---
# START_DATE = "2025-01-01"
# END_DATE = "2025-03-31"

# # --- Site Identifiers for the .WTH file ---
# SITE_NAME = "CHENNAI"
# INSTITUTION = "NASA"
# SITE_CODE = "IN01"      # 4-character code for the site

# # ==============================================================================
# # --- HELPER FUNCTIONS (Integrated from other scripts) ---
# # ==============================================================================

# def _fetch_co2_data(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
#     """
#     Fetches and caches daily CO2 data from NOAA's Mauna Loa Observatory.
#     """
#     DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
#     req_start_date = pd.to_datetime(start_date_str)
#     req_end_date = pd.to_datetime(end_date_str)
    
#     download_needed = not os.path.exists(filename)

#     if not download_needed:
#         try:
#             # Check if existing file covers the required date range
#             df_check = pd.read_csv(filename, comment="#", sep=r'\s+', usecols=[0, 1, 2], names=["yr", "mon", "day"])
#             date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
#             df_check['date'] = pd.to_datetime(df_check[['yr', 'mon', 'day']].rename(columns=date_cols))
#             if req_start_date < df_check['date'].min() or req_end_date > df_check['date'].max():
#                 download_needed = True
#         except (pd.errors.EmptyDataError, KeyError):
#             download_needed = True

#     if download_needed:
#         print("⬇️  Downloading latest CO2 data from NOAA...")
#         try:
#             response = requests.get(DATA_URL, timeout=15)
#             response.raise_for_status()
#             with open(filename, 'w', encoding='utf-8') as file:
#                 file.write(response.text)
#             print(f"✅ CO2 data saved to {filename}")
#         except requests.exceptions.RequestException as e:
#             print(f"❌ ERROR: Could not fetch CO2 data. {e}")
#             return pd.DataFrame()

#     try:
#         df = pd.read_csv(filename, comment="#", sep=r'\s+', names=["yr", "mon", "day", "decimal", "ppm"])
#         date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
#         df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns=date_cols))
#         mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
#         return df.loc[mask][['date', 'ppm']].reset_index(drop=True)
#     except FileNotFoundError:
#         print(f"❌ ERROR: Could not find {filename} after attempting download.")
#         return pd.DataFrame()


# def _fetch_air_quality_data(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
#     """
#     Fetches daily average air quality data (specifically O3) from OpenWeatherMap.
#     """
#     if not api_key or api_key == "YOUR_API_KEY_HERE":
#         print("⚠️  WARNING: OpenWeatherMap API key not provided. Skipping Ozone fetch.")
#         return pd.DataFrame()

#     try:
#         start_dt = datetime.strptime(start_str, "%Y-%m-%d")
#         end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
#         start_ts = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
#         end_ts = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

#         url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
#                f"lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}")
#         print(url)
#         print("⬇️  Downloading daily Ozone data from OpenWeatherMap...")
#         response = requests.get(url)
#         response.raise_for_status()
#         data = response.json()

#         daily_data = defaultdict(lambda: defaultdict(list))
#         if 'list' in data:
#             for entry in data['list']:
#                 local_date = datetime.fromtimestamp(entry['dt'], tz=timezone.utc).date()
#                 if 'o3' in entry['components']:
#                     daily_data[local_date]['o3'].append(entry['components']['o3'])
        
#         processed_data = []
#         for date, components in sorted(daily_data.items()):
#             if components['o3']:
#                 avg_o3 = sum(components['o3']) / len(components['o3'])
#                 processed_data.append({'date': pd.to_datetime(date), 'o3_ug_m3': avg_o3})
        
#         print("✅ Ozone data processed.")
#         print(processed_data)
#         return pd.DataFrame(processed_data)

#     except requests.exceptions.HTTPError as e:
#         print(f"❌ ERROR fetching air quality: {e.response.status_code} {e.response.reason}. Check your API key.")
#         return pd.DataFrame()
#     except Exception as e:
#         print(f"❌ An error occurred while fetching air quality data: {e}")
#         return pd.DataFrame()

# # ==============================================================================
# # --- MAIN FUNCTION ---
# # ==============================================================================

# def fetch_weather_data(lat, lon, start, end, owm_api_key, site_name="SITE", inst="NASA", insi="XXXX", elev=None):
#     """
#     Fetches weather, CO2, and Ozone data to create a DSSAT-compatible .WTH file.
    
#     Args:
#         lat (float): Latitude of the site.
#         lon (float): Longitude of the site.
#         start (str): Start date in 'YYYY-MM-DD' format.
#         end (str): End date in 'YYYY-MM-DD' format.
#         owm_api_key (str): API key for OpenWeatherMap to fetch Ozone data.
#         site_name (str): Name of the site for the file header.
#         inst (str): Institution name for the file header.
#         insi (str): 4-character site code.
#         elev (float, optional): Elevation in meters. Defaults to 100 if None.
#     """
#     # --- 1. Fetch Primary Weather Data from NASA POWER ---
#     print("⬇️  Downloading primary weather data from NASA POWER...")
#     start_fmt = pd.to_datetime(start).strftime("%Y%m%d")
#     end_fmt = pd.to_datetime(end).strftime("%Y%m%d")
#     url = "https://power.larc.nasa.gov/api/temporal/daily/point"
#     params = {
#         "start": start_fmt, "end": end_fmt, "latitude": lat, "longitude": lon,
#         "community": "AG",
#         "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
#         "format": "JSON"
#     }
#     response = requests.get(url, params=params).json()
#     data = response["properties"]["parameter"]
    
#     date_keys = [k for k in data[list(data.keys())[0]].keys() if k.isdigit()]
#     df = pd.DataFrame({param: {date: data[param][date] for date in date_keys} for param in data})
#     df = df.reset_index().rename(columns={"index": "date"})
#     df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
#     print("✅ NASA POWER data processed.")

#     # --- 2. Fetch Dynamic CO2 and Ozone Data ---
#     co2_df = _fetch_co2_data(start, end)
#     # aq_df = _fetch_air_quality_data(owm_api_key, lat, lon, start, end)
#     aq_df = get_daily_average_air_quality(owm_api_key, lat, lon, start, end)
#     print(aq_df)
    
#     # --- 3. Merge and Process DataFrames ---
#     if not co2_df.empty:
#         df = pd.merge(df, co2_df, on='date', how='left')
#     if not aq_df.empty:
#         df = pd.merge(df, aq_df, on='date', how='left')

#     # Rename columns for DSSAT
#     df.rename(columns={
#         "ALLSKY_SFC_SW_DWN": "srad", "T2M_MAX": "tmax", "T2M_MIN": "tmin",
#         "T2MDEW": "dewp", "PRECTOTCORR": "rain", "WS2M": "wind",
#         "RH2M": "rhum", "EVPTRNS": "evap"
#     }, inplace=True)

#     # Unit conversions and Calculations
#     df["wind"] = df["wind"] * 86.4  # m/s → km/day
#     df["month"] = df["date"].dt.month
#     monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
#     df["par"] = df["month"].map(monthly_srad_avg)
#     df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))

#     # --- 4. Assign Dynamic CO2 and Ozone with Fallbacks ---
#     # Assign CO2 (ppm), fallback to 421.0 if API data is missing for a day
#     df['dco2'] = df.get('ppm', pd.Series(index=df.index)).fillna(421.0)
    
#     # Convert Ozone from µg/m³ to ppb (1 µg/m³ ≈ 0.51 ppb)
#     # Fallback to 50.0 if API data is missing for a day
#     if 'o3_ug_m3' in df.columns:
#         df['ozon7'] = (df['o3_ug_m3'] * 0.51).fillna(50.0)
#     else:
#         df['ozon7'] = 50.0

#     # --- 5. Finalize DataFrame for DSSAT Output ---
#     df["YRDOY"] = df["date"].dt.strftime("%y%j")
#     tav = ((df["tmax"] + df["tmin"]) / 2).mean()
#     amp = (df["tmax"].mean() - df["tmin"].mean())
    
#     df_out = df[["YRDOY", "srad", "tmax", "tmin", "rain", "rhum", "wind",
#                  "dewp", "par", "vapr", "dco2", "ozon7", "evap"]]

#     # --- 6. Save .WTH File ---
#     output_dir = "weather"
#     os.makedirs(output_dir, exist_ok=True)
#     start_yrdoy = df_out["YRDOY"].iloc[0]
#     wth_file = os.path.join(output_dir, f"{insi}_{start_yrdoy}.WTH")

#     with open(wth_file, "w") as f:
#         f.write(f"*WEATHER DATA : {site_name},{inst}\n\n")
#         f.write("@ INSI      LAT     LONG   ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
#         f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev is not None else 100:5.1f} {tav:5.1f} {amp:5.1f}   2.0   2.0 {df['dco2'].mean():6.1f}\n\n")
#         f.write("@  DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
#         for _, row in df_out.iterrows():
#             f.write(f"{int(row.YRDOY):7d} "
#                     f"{row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
#                     f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} "
#                     f"{row.dewp:5.1f} {row.par:6.1f} {row.vapr:5.2f} "
#                     f"{row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")

#     print(f"\n🎉 Success! Weather file saved at: {wth_file}")
#     return df_out

# # ==============================================================================
# # --- ▶️ SCRIPT EXECUTION ---
# # ==============================================================================
# if __name__ == "__main__":
#     # Call the main function with the configured parameters
#     final_df = fetch_weather_data(
#         lat=LATITUDE,
#         lon=LONGITUDE,
#         start=START_DATE,
#         end=END_DATE,
#         owm_api_key=OWM_API_KEY,
#         site_name=SITE_NAME,
#         inst=INSTITUTION,
#         insi=SITE_CODE,
#         elev=ELEVATION
#     )
    
#     # Display the head of the final DataFrame
#     if final_df is not None:
#         print("\n--- Final DataFrame Head ---")
#         print(final_df.head())

In [74]:
import requests
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import statistics
import pandas as pd # Added pandas import
import os
from datetime import datetime

def get_co2(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
    """
    Fetches daily CO2 data for a given date range, caching the data locally.

    It checks if the data file exists and if the requested date range is
    covered. It only downloads a fresh copy if the file is missing or the
    date range is not fully available.

    Args:
        start_date_str (str): The start date in 'YYYY-MM-DD' format.
        end_date_str (str): The end date in 'YYYY-MM-DD' format.
        filename (str): The local filename to use for caching.

    Returns:
        pd.DataFrame: A DataFrame containing the CO2 data for the specified range,
                      or an empty DataFrame if data cannot be retrieved.
    """
    DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
    
    # Convert input strings to datetime objects for comparison
    req_start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    req_end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    download_needed = False
    
    # 1. Check if the file exists and if the date range is covered
    if os.path.exists(filename):
        print(f"📄 Local file '{filename}' found.")
        try:
            # Read only essential columns to check date range quickly
            local_df_check = pd.read_csv(
                filename,
                comment="#",
                sep=r'\s+',
                usecols=[0, 1, 2],
                names=["yr", "mon", "day"]
            )
            # Rename columns for pd.to_datetime compatibility
            date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
            local_df_check['date'] = pd.to_datetime(local_df_check[['yr', 'mon', 'day']].rename(columns=date_cols))
            
            local_min_date = local_df_check['date'].min()
            local_max_date = local_df_check['date'].max()
            print(f"   - Local data covers: {local_min_date.date()} to {local_max_date.date()}")

            # 2. Decide if a new download is necessary
            if req_start_date < local_min_date or req_end_date > local_max_date:
                print("   - Requested date range is outside local data. A new download is required.")
                download_needed = True
            else:
                print("   - Local data is sufficient for the requested range.")
                
        except (pd.errors.EmptyDataError, KeyError, IndexError):
            print("   - Local file seems corrupt or empty. Re-downloading.")
            download_needed = True
    else:
        print(f"📄 Local file '{filename}' not found. Download is required.")
        download_needed = True

    # 3. Download if needed
    if download_needed:
        try:
            print(f"⬇️ Downloading latest data from {DATA_URL}...")
            response = requests.get(DATA_URL, timeout=15)
            response.raise_for_status()

            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"✅ Download complete. Data saved to {filename}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Error: Could not fetch the URL. {e}")
            return pd.DataFrame()

    # 4. Read the definitive local file and filter it
    print(f"Reading and processing {filename}...")
    try:
        df = pd.read_csv(
            filename,
            comment="#",
            sep=r'\s+',
            names=["yr", "mon", "day", "decimal", "ppm"]
        )
        # Rename columns for pd.to_datetime compatibility
        date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
        df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns=date_cols))
        
        mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
        filtered_df = df.loc[mask].reset_index(drop=True)
        print("✅ Processing complete.")
        return pd.DataFrame(filtered_df[['date', 'ppm']])

    except FileNotFoundError:
        print(f"❌ Error: Could not find {filename} after attempting to download.")
        return pd.DataFrame()


def get_o3(api_key, lat, lon, start_str, end_str):
    """
    Fetches historical air quality data, calculates daily averages, 
    and returns the results as a pandas DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame containing the daily air quality summary,
                      or None if an error occurs or no data is found.
    """
    if api_key == "YOUR_API_KEY_HERE":
        print("🛑 Error: Please replace 'YOUR_API_KEY_HERE' with your actual OpenWeatherMap API key.")
        return None

    aqi_map = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}
    ist_tz = timezone(timedelta(hours=5, minutes=30))

    try:
        # Step 1: Convert dates to UNIX timestamps
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        start_timestamp = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_timestamp = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        # Step 2: Build the API URL and make the request
        api_url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
                   f"lat={lat}&lon={lon}&start={start_timestamp}&end={end_timestamp}&appid={api_key}")
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        # Step 3: Aggregate hourly data by date
        daily_data = defaultdict(lambda: defaultdict(list))

        if 'list' not in data or not data['list']:
            print("No data found for the specified location and date range.")
            return None
        
        for entry in data['list']:
            local_date = datetime.fromtimestamp(entry['dt'], tz=ist_tz).date()
            daily_data[local_date]['aqi'].append(entry['main']['aqi'])
            for component, value in entry['components'].items():
                daily_data[local_date][component].append(value)

        # Step 4: Process aggregated data and prepare for DataFrame
        records = [] # This list will hold the data for our DataFrame
        
        for date in sorted(daily_data.keys()):
            day = daily_data[date]
            
            # Find the most frequent (mode) AQI for the day
            try:
                modal_aqi_index = statistics.mode(day['aqi'])
            except statistics.StatisticsError: # Handles cases with no unique mode
                modal_aqi_index = day['aqi'][0] # Fallback to the first value
            
            modal_aqi_text = aqi_map.get(modal_aqi_index, "Unknown")
            
            # Create a dictionary for the current row
            row_data = {
                'date': date.strftime('%Y-%m-%d'),
                'OZON7': sum(day['o3']) / len(day['o3']),
            }
            records.append(row_data)

        if not records:
            print("No data was processed to create a DataFrame.")
            return None

        # Step 5: Create and return the DataFrame
        df = pd.DataFrame(records)
        return df

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print(f"HTTP Error 401: Unauthorized. Please check if your API key is correct and active.")
        else:
            print(f"An HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"A network error occurred: {e}")
    except (KeyError, IndexError, ZeroDivisionError) as e:
        print(f"Error processing data: {e}. The API response may be incomplete or in an unexpected format.")
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")
    
    return None # Return None if any exception occurred




# # --- Call the function and display the DataFrame ---
# if __name__ == "__main__":
#     API_KEY = "eddbc7c0c9e63e225934add809624c6e"
#     LATITUDE = 11.202263   # A specific location
#     LONGITUDE = 76.96184
#     # Specify the date range (Format: YYYY-MM-DD)
#     START_DATE = "2024-09-01"
#     END_DATE = "2024-10-31"

#     co2 = get_co2(START_DATE, END_DATE)
#     o3 = get_o3(API_KEY, lat=LATITUDE, lon=LONGITUDE, start_str=START_DATE, end_str=END_DATE)
#     merged_df = pd.concat(o3, co2, join='inner')
    
#     print(merged_df)
    


In [78]:
import requests
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import statistics
import pandas as pd
import os

# --- (The get_co2 and get_o3 functions remain exactly the same) ---
def get_co2(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
    """
    Fetches daily CO2 data for a given date range, caching the data locally.
    It checks if the data file exists and if the requested date range is
    covered. It only downloads a fresh copy if the file is missing or the
    date range is not fully available.
    Args:
        start_date_str (str): The start date in 'YYYY-MM-DD' format.
        end_date_str (str): The end date in 'YYYY-MM-DD' format.
        filename (str): The local filename to use for caching.
    Returns:
        pd.DataFrame: A DataFrame containing the CO2 data for the specified range,
                      or an empty DataFrame if data cannot be retrieved.
    """
    DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
    
    # Convert input strings to datetime objects for comparison
    req_start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    req_end_date = datetime.strptime(end_date_str, '%Y-%m-%d')
    
    download_needed = False
    
    # 1. Check if the file exists and if the date range is covered
    if os.path.exists(filename):
        print(f"📄 Local file '{filename}' found.")
        try:
            # Read only essential columns to check date range quickly
            local_df_check = pd.read_csv(
                filename,
                comment="#",
                sep=r'\s+',
                usecols=[0, 1, 2],
                names=["yr", "mon", "day"]
            )
            # Rename columns for pd.to_datetime compatibility
            date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
            local_df_check['date'] = pd.to_datetime(local_df_check[['yr', 'mon', 'day']].rename(columns=date_cols))
            
            local_min_date = local_df_check['date'].min()
            local_max_date = local_df_check['date'].max()
            print(f"   - Local data covers: {local_min_date.date()} to {local_max_date.date()}")

            # 2. Decide if a new download is necessary
            if req_start_date < local_min_date or req_end_date > local_max_date:
                print("   - Requested date range is outside local data. A new download is required.")
                download_needed = True
            else:
                print("   - Local data is sufficient for the requested range.")
                
        except (pd.errors.EmptyDataError, KeyError, IndexError):
            print("   - Local file seems corrupt or empty. Re-downloading.")
            download_needed = True
    else:
        print(f"📄 Local file '{filename}' not found. Download is required.")
        download_needed = True

    # 3. Download if needed
    if download_needed:
        try:
            print(f"⬇️ Downloading latest data from {DATA_URL}...")
            response = requests.get(DATA_URL, timeout=15)
            response.raise_for_status()

            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"✅ Download complete. Data saved to {filename}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Error: Could not fetch the URL. {e}")
            return pd.DataFrame()

    # 4. Read the definitive local file and filter it
    print(f"Reading and processing {filename}...")
    try:
        df = pd.read_csv(
            filename,
            comment="#",
            sep=r'\s+',
            names=["yr", "mon", "day", "decimal", "ppm"]
        )
        # Rename columns for pd.to_datetime compatibility
        date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
        df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns=date_cols))
        
        mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
        filtered_df = df.loc[mask].reset_index(drop=True)
        print("✅ Processing complete.")
        return pd.DataFrame(filtered_df[['date', 'ppm']])

    except FileNotFoundError:
        print(f"❌ Error: Could not find {filename} after attempting to download.")
        return pd.DataFrame()


def get_o3(api_key, lat, lon, start_str, end_str):
    """
    Fetches historical air quality data, calculates daily averages, 
    and returns the results as a pandas DataFrame.
    
    Returns:
        pd.DataFrame: A DataFrame containing the daily air quality summary,
                      or None if an error occurs or no data is found.
    """
    if api_key == "YOUR_API_KEY_HERE":
        print("🛑 Error: Please replace 'YOUR_API_KEY_HERE' with your actual OpenWeatherMap API key.")
        return None

    aqi_map = {1: "Good", 2: "Fair", 3: "Moderate", 4: "Poor", 5: "Very Poor"}
    ist_tz = timezone(timedelta(hours=5, minutes=30))

    try:
        # Step 1: Convert dates to UNIX timestamps
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        start_timestamp = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_timestamp = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        # Step 2: Build the API URL and make the request
        api_url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
                   f"lat={lat}&lon={lon}&start={start_timestamp}&end={end_timestamp}&appid={api_key}")
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        # Step 3: Aggregate hourly data by date
        daily_data = defaultdict(lambda: defaultdict(list))

        if 'list' not in data or not data['list']:
            print("No data found for the specified location and date range.")
            return None
        
        for entry in data['list']:
            local_date = datetime.fromtimestamp(entry['dt'], tz=ist_tz).date()
            daily_data[local_date]['aqi'].append(entry['main']['aqi'])
            for component, value in entry['components'].items():
                daily_data[local_date][component].append(value)

        # Step 4: Process aggregated data and prepare for DataFrame
        records = [] # This list will hold the data for our DataFrame
        
        for date in sorted(daily_data.keys()):
            day = daily_data[date]
            
            # Find the most frequent (mode) AQI for the day
            try:
                modal_aqi_index = statistics.mode(day['aqi'])
            except statistics.StatisticsError: # Handles cases with no unique mode
                modal_aqi_index = day['aqi'][0] # Fallback to the first value
            
            modal_aqi_text = aqi_map.get(modal_aqi_index, "Unknown")
            
            # Create a dictionary for the current row
            row_data = {
                'date': date.strftime('%Y-%m-%d'),
                'OZON7': sum(day['o3']) / len(day['o3']),
            }
            records.append(row_data)

        if not records:
            print("No data was processed to create a DataFrame.")
            return None

        # Step 5: Create and return the DataFrame
        df = pd.DataFrame(records)
        return df

    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 401:
            print(f"HTTP Error 401: Unauthorized. Please check if your API key is correct and active.")
        else:
            print(f"An HTTP error occurred: {e}")
    except requests.exceptions.RequestException as e:
        print(f"A network error occurred: {e}")
    except (KeyError, IndexError, ZeroDivisionError) as e:
        print(f"Error processing data: {e}. The API response may be incomplete or in an unexpected format.")
    except ValueError:
        print("Error: Invalid date format. Please use YYYY-MM-DD.")
    
    return None # Return None if any exception occurred

# --- Call the function and display the DataFrame ---
if __name__ == "__main__":
    API_KEY = "eddbc7c0c9e63e225934add809624c6e"
    LATITUDE = 11.202263   # A specific location
    LONGITUDE = 76.96184
    # Specify the date range (Format: YYYY-MM-DD)
    START_DATE = "2024-09-01"
    END_DATE = "2024-10-31"

    co2_df = get_co2(START_DATE, END_DATE)
    o3_df = get_o3(API_KEY, lat=LATITUDE, lon=LONGITUDE, start_str=START_DATE, end_str=END_DATE)

    # Make sure both DataFrames were created successfully before trying to merge
    if o3_df is not None and not co2_df.empty:
        # Fix 1: Ensure date columns have the same data type
        o3_df['date'] = pd.to_datetime(o3_df['date'])
        
        # Fix 2: Use pd.merge to join on the 'date' column
        merged_df = pd.merge(o3_df, co2_df, on='date', how='inner')
    
        print("\n--- Merged DataFrame ---")
        print(merged_df)
    else:
        print("\nCould not create one or both DataFrames. Merge aborted.")

📄 Local file 'co2_daily_mlo.txt' found.
   - Local data covers: 1974-05-19 to 2025-09-12
   - Local data is sufficient for the requested range.
Reading and processing co2_daily_mlo.txt...
✅ Processing complete.

--- Merged DataFrame ---
         date      OZON7     ppm
0  2024-09-02  19.846667  422.74
1  2024-09-03  26.442083  422.78
2  2024-09-05  24.184583  421.40
3  2024-09-06  24.729167  422.10
4  2024-09-07  28.275417  422.64
5  2024-09-08  24.736667  422.40
6  2024-09-10  27.517222  421.75
7  2024-09-14  25.957500  422.04
8  2024-09-15  29.061667  422.13
9  2024-09-16  35.970833  421.90
10 2024-09-17  40.351667  421.88
11 2024-09-18  33.232917  422.03
12 2024-09-19  45.832083  421.73
13 2024-09-20  44.033333  421.99
14 2024-09-21  40.388750  422.19
15 2024-09-26  27.760833  421.63
16 2024-09-27  24.499167  421.79
17 2024-09-30  53.272083  421.95
18 2024-10-03  30.562917  421.88
19 2024-10-04  32.187500  422.27
20 2024-10-05  29.377500  421.69
21 2024-10-06  34.821250  422.27
22 2

In [ ]:
import requests
import pandas as pd
import os
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import statistics
import concurrent.futures # Added for concurrency

def get_co2(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
    """
    Optimized: Fetches daily CO2 data, reading the local file only once.
    """
    DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
    req_start_date = pd.to_datetime(start_date_str)
    req_end_date = pd.to_datetime(end_date_str)
    
    download_needed = True
    df = pd.DataFrame()

    # Optimization 1: Read the local file only ONCE.
    if os.path.exists(filename):
        print(f"📄 Local file '{filename}' found. Reading it to check date range...")
        try:
            df = _read_noaa_file(filename)
            local_min_date = df['date'].min()
            local_max_date = df['date'].max()
            print(f"   - Local data covers: {local_min_date.date()} to {local_max_date.date()}")
            
            # If local data is sufficient, no download is needed.
            if req_start_date >= local_min_date and req_end_date <= local_max_date:
                print("   - Local data is sufficient for the requested range.")
                download_needed = False
            else:
                print("   - Requested date range is outside local data. A new download is required.")
        except (pd.errors.EmptyDataError, KeyError, IndexError):
            print("   - Local file seems corrupt or empty. Re-downloading.")

    if download_needed:
        try:
            print(f"⬇️ Downloading latest data from {DATA_URL}...")
            response = requests.get(DATA_URL, timeout=15)
            response.raise_for_status()
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"✅ Download complete. Data saved to {filename}")
            # Read the newly downloaded file
            df = _read_noaa_file(filename)
        except requests.exceptions.RequestException as e:
            print(f"❌ Error: Could not fetch the URL. {e}")
            return pd.DataFrame() # Return empty DataFrame on failure
        except FileNotFoundError:
            print(f"❌ Error: Could not find or read {filename} after attempting to download.")
            return pd.DataFrame()

    if not df.empty:
        mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
        print("✅ CO2 processing complete.")
        return df.loc[mask, ['date', 'ppm']].reset_index(drop=True)
    
    return pd.DataFrame()

def _read_noaa_file(filepath: str) -> pd.DataFrame:
    """Helper function to read and parse the NOAA data file."""
    df = pd.read_csv(
        filepath,
        comment="#",
        sep=r'\s+',
        names=["yr", "mon", "day", "decimal", "ppm"]
    )
    date_cols = {'yr': 'year', 'mon': 'month', 'day': 'day'}
    df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns=date_cols))
    return df

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """
    Optimized: Returns the 'date' column with the correct datetime type directly.
    """
    if api_key == "YOUR_API_KEY_HERE":
        print("🛑 Error: Please replace 'YOUR_API_KEY_HERE' with your actual OpenWeatherMap API key.")
        return pd.DataFrame()

    ist_tz = timezone(timedelta(hours=5, minutes=30))

    try:
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        start_timestamp = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_timestamp = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        api_url = (f"http://api.openweathermap.org/data/2.5/air_pollution/history?"
                   f"lat={lat}&lon={lon}&start={start_timestamp}&end={end_timestamp}&appid={api_key}")
        
        print(f"⬇️ Fetching O3 data from OpenWeatherMap API...")
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        if 'list' not in data or not data['list']:
            print("No O3 data found for the specified location and date range.")
            return pd.DataFrame()
        
        daily_data = defaultdict(list)
        for entry in data['list']:
            local_date = datetime.fromtimestamp(entry['dt'], tz=ist_tz).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = []
        for date, o3_values in sorted(daily_data.items()):
            # Optimization 3: Keep 'date' as a datetime.date object
            row_data = {
                'date': date,
                'OZON7': sum(o3_values) / len(o3_values)
            }
            records.append(row_data)

        if not records:
            return pd.DataFrame()
            
        print("✅ O3 processing complete.")
        # pandas will automatically convert the 'date' column to datetime64[ns]
        return pd.DataFrame(records)

    except requests.exceptions.RequestException as e:
        print(f"❌ A network error occurred while fetching O3 data: {e}")
    except (KeyError, IndexError, ZeroDivisionError) as e:
        print(f"❌ Error processing O3 data: {e}.")
    
    return pd.DataFrame()


# --- Main Execution Block ---
if __name__ == "__main__":
    API_KEY = "eddbc7c0c9e63e225934add809624c6e"
    LATITUDE = 11.202263
    LONGITUDE = 76.96184
    START_DATE = "2024-09-01"
    END_DATE = "2024-10-31"

    co2_df = pd.DataFrame()
    o3_df = pd.DataFrame()
    
    # --- Concurrent Data Fetch (no changes here) ---
    print("--- Starting Concurrent Data Fetch ---")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_co2 = executor.submit(get_co2, START_DATE, END_DATE)
        future_o3 = executor.submit(get_o3, API_KEY, LATITUDE, LONGITUDE, START_DATE, END_DATE)

        co2_df = future_co2.result()
        o3_df = future_o3.result()
    print("--- Concurrent Data Fetch Complete ---\n")

    # Merge the results
    if not o3_df.empty and not co2_df.empty:
        
        # --- FIX: Ensure date columns have the same datetime64[ns] type ---
        o3_df['date'] = pd.to_datetime(o3_df['date'])
        
        # Now the merge will work correctly
        merged_df = pd.merge(o3_df, co2_df, on='date', how='inner')
    
        print("\n--- Merged DataFrame ---")
        if merged_df.empty:
            print("No matching dates found between the two datasets.")
        else:
            print(merged_df)
    else:
        print("\nCould not create one or both DataFrames. Merge aborted.")

--- Starting Concurrent Data Fetch ---
📄 Local file 'co2_daily_mlo.txt' found. Reading it to check date range...
⬇️ Fetching O3 data from OpenWeatherMap API...
   - Local data covers: 1974-05-19 to 2025-09-12
   - Local data is sufficient for the requested range.
✅ CO2 processing complete.
✅ O3 processing complete.
--- Concurrent Data Fetch Complete ---


--- Merged DataFrame ---
         date      OZON7     ppm
0  2024-09-02  19.846667  422.74
1  2024-09-03  26.442083  422.78
2  2024-09-05  24.184583  421.40
3  2024-09-06  24.729167  422.10
4  2024-09-07  28.275417  422.64
5  2024-09-08  24.736667  422.40
6  2024-09-10  27.517222  421.75
7  2024-09-14  25.957500  422.04
8  2024-09-15  29.061667  422.13
9  2024-09-16  35.970833  421.90
10 2024-09-17  40.351667  421.88
11 2024-09-18  33.232917  422.03
12 2024-09-19  45.832083  421.73
13 2024-09-20  44.033333  421.99
14 2024-09-21  40.388750  422.19
15 2024-09-26  27.760833  421.63
16 2024-09-27  24.499167  421.79
17 2024-09-30  53.27208

In [83]:
import requests
import pandas as pd
import numpy as np # Added for calculations
import os
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import concurrent.futures

# ==============================================================================
# DATA FETCHING HELPERS (CO2 and O3)
# These functions remain largely unchanged but will be called by fetch_weather.
# ==============================================================================

def get_co2(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
    """Optimized: Fetches daily CO2 data, reading the local file only once."""
    DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
    req_start_date = pd.to_datetime(start_date_str)
    req_end_date = pd.to_datetime(end_date_str)
    
    download_needed = True
    df = pd.DataFrame()

    if os.path.exists(filename):
        print(f"📄 Local file '{filename}' found. Checking date range...")
        try:
            df = _read_noaa_file(filename)
            local_min_date = df['date'].min()
            local_max_date = df['date'].max()
            if req_start_date >= local_min_date and req_end_date <= local_max_date:
                print("   - Local data is sufficient.")
                download_needed = False
            else:
                print("   - Local data is outdated. Download required.")
        except (pd.errors.EmptyDataError, KeyError, IndexError):
            print("   - Local file corrupt. Re-downloading.")

    if download_needed:
        try:
            print(f"⬇️ Downloading CO2 data from NOAA...")
            response = requests.get(DATA_URL, timeout=15)
            response.raise_for_status()
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(response.text)
            print(f"✅ CO2 Download complete.")
            df = _read_noaa_file(filename)
        except requests.exceptions.RequestException as e:
            print(f"❌ Error fetching CO2 URL: {e}")
            return pd.DataFrame()

    if not df.empty:
        mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
        print("✅ CO2 processing complete.")
        return df.loc[mask, ['date', 'ppm']].reset_index(drop=True)
    
    return pd.DataFrame()

def _read_noaa_file(filepath: str) -> pd.DataFrame:
    """Helper function to read and parse the NOAA data file."""
    df = pd.read_csv(filepath, comment="#", sep=r'\s+', names=["yr", "mon", "day", "decimal", "ppm"])
    df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns={'yr': 'year', 'mon': 'month', 'day': 'day'}))
    return df

def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
    """Optimized: Fetches O3 data and returns a clean DataFrame."""
    if not api_key or api_key == "YOUR_API_KEY_HERE":
        print("🛑 O3 Fetching Skipped: Invalid OpenWeatherMap API key.")
        return pd.DataFrame()

    try:
        start_dt = datetime.strptime(start_str, "%Y-%m-%d")
        end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
        start_ts = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
        end_ts = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

        api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
        print(f"⬇️ Fetching O3 data from OpenWeatherMap API...")
        response = requests.get(api_url)
        response.raise_for_status()
        data = response.json()

        if 'list' not in data or not data['list']:
            print("No O3 data found for the specified location and date range.")
            return pd.DataFrame()
        
        daily_data = defaultdict(list)
        for entry in data['list']:
            local_date = datetime.fromtimestamp(entry['dt'], tz=timezone(timedelta(hours=5, minutes=30))).date()
            daily_data[local_date].append(entry['components'].get('o3', 0))

        records = [{'date': date, 'OZON7': sum(vals) / len(vals)} for date, vals in sorted(daily_data.items())]
        
        if not records:
            return pd.DataFrame()
        
        print("✅ O3 processing complete.")
        return pd.DataFrame(records)

    except requests.exceptions.RequestException as e:
        print(f"❌ Network error fetching O3 data: {e}")
    except (KeyError, IndexError, ZeroDivisionError) as e:
        print(f"❌ Error processing O3 data: {e}.")
    
    return pd.DataFrame()

# ==============================================================================
# MAIN ORCHESTRATOR FUNCTION
# This function now handles everything: fetching all data, merging, and writing.
# ==============================================================================

def create_weather_file(api_key, lat, lon, start_date, end_date, site_name="SITE", insi="XXXX", elev=None):
    """
    Fetches weather, CO2, and O3 data concurrently, merges them,
    and writes a DSSAT-compatible .WTH file.
    """
    # --- 1. CONCURRENTLY FETCH ALL DATA SOURCES ---
    weather_df, co2_df, o3_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    print("--- Starting Concurrent Data Fetch ---")
    with concurrent.futures.ThreadPoolExecutor() as executor:
        future_weather = executor.submit(_fetch_nasa_weather, lat, lon, start_date, end_date)
        future_co2 = executor.submit(get_co2, start_date, end_date)
        future_o3 = executor.submit(get_o3, api_key, lat, lon, start_date, end_date)
        
        weather_df = future_weather.result()
        co2_df = future_co2.result()
        o3_df = future_o3.result()
    print("--- Concurrent Data Fetch Complete ---\n")

    if weather_df.empty:
        print("❌ Critical Error: Could not fetch base weather data from NASA. Aborting.")
        return

    # --- 2. MERGE AND PROCESS DATA ---
    # Start with the base weather data
    df = weather_df
    
    # Merge CO2 data if available
    if not co2_df.empty:
        df = pd.merge(df, co2_df, on='date', how='left')
        df = df.rename(columns={"ppm": "dco2"})
    
    # Merge O3 data if available
    if not o3_df.empty:
        # Ensure date types match before merging
        o3_df['date'] = pd.to_datetime(o3_df['date'])
        df = pd.merge(df, o3_df, on='date', how='left')
        df = df.rename(columns={"OZON7": "ozon7"})

    # Use fillna() for robust handling of missing data
    df['dco2'] = df.get('dco2', 421.0).fillna(421.0)
    df['ozon7'] = df.get('ozon7', 50).fillna(50)

    # --- 3. PERFORM CALCULATIONS ---
    df["wind"] = df["wind"] * 86.4  # m/s → km/day
    df["month"] = df["date"].dt.month
    monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
    df["par"] = df["month"].map(monthly_srad_avg)
    df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
    df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
    tav = ((df["tmax"] + df["tmin"]) / 2).mean()
    amp = (df["tmax"].mean() - df["tmin"].mean())

    # --- 4. PREPARE AND WRITE THE .WTH FILE ---
    df_out = df[["YRDOY","srad","tmax","tmin","rain","rhum","wind",
                 "dewp","par","vapr","dco2","ozon7","evap"]]

    output_dir = "weather"
    os.makedirs(output_dir, exist_ok=True)
    start_yrdoy = df_out["YRDOY"].iloc[0]
    wth_file = os.path.join(output_dir, f"{insi}_{start_yrdoy}.WTH")

    with open(wth_file, "w") as f:
        f.write(f"*WEATHER DATA : {site_name},{'NASA'}\n\n")
        f.write("@ INSI      LAT     LONG   ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
        f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else 100:5.1f}  {tav:5.1f}  {amp:5.1f}   2.0   2.0  {df['dco2'].mean():6.1f}\n\n")
        f.write("@  DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
        for _, row in df_out.iterrows():
            f.write(f"{int(row.YRDOY):7d} {row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
                    f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} {row.dewp:5.1f} "
                    f"{row.par:6.1f} {row.vapr:5.2f} {row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")

    print(f"✅ Weather file successfully saved at: {wth_file}")
    return df_out

def _fetch_nasa_weather(lat, lon, start, end):
    """Helper function dedicated to fetching NASA POWER data."""
    print("⬇️ Fetching base weather data from NASA POWER...")
    try:
        url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        params = {
            "start": pd.to_datetime(start).strftime("%Y%m%d"),
            "end": pd.to_datetime(end).strftime("%Y%m%d"),
            "latitude": lat, "longitude": lon, "community": "AG",
            "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
            "format": "JSON"
        }
        response = requests.get(url, params=params).json()
        data = response["properties"]["parameter"]
        
        df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
        df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        
        df = df.rename(columns={
            "ALLSKY_SFC_SW_DWN":"srad", "T2M_MAX":"tmax", "T2M_MIN":"tmin", "T2MDEW":"dewp",
            "PRECTOTCORR":"rain", "WS2M":"wind", "RH2M":"rhum", "EVPTRNS":"evap"
        })
        print("✅ NASA Weather processing complete.")
        return df
    except Exception as e:
        print(f"❌ Error fetching NASA weather data: {e}")
        return pd.DataFrame()

# ==============================================================================
# SCRIPT EXECUTION
# ==============================================================================
if __name__ == "__main__":
    # --- Configuration ---
    API_KEY = "eddbc7c0c9e63e225934add809624c6e" # Your OpenWeatherMap API Key
    LATITUDE = 13.08
    LONGITUDE = 80.27
    START_DATE = "2024-01-01"
    END_DATE = "2024-12-31"
    
    # --- Execute the Main Function ---
    final_data = create_weather_file(
        api_key=API_KEY,
        lat=LATITUDE,
        lon=LONGITUDE,
        start_date=START_DATE,
        end_date=END_DATE,
        site_name="CHENNAI",
        insi="IN01",
        elev=6
    )

    if final_data is not None:
        print("\n--- Final Data Head ---")
        print(final_data.head())

--- Starting Concurrent Data Fetch ---
⬇️ Fetching base weather data from NASA POWER...
📄 Local file 'co2_daily_mlo.txt' found. Checking date range...
⬇️ Fetching O3 data from OpenWeatherMap API...
   - Local data is sufficient.
✅ CO2 processing complete.
✅ NASA Weather processing complete.
✅ O3 processing complete.
--- Concurrent Data Fetch Complete ---

✅ Weather file successfully saved at: weather\IN01_24001.WTH

--- Final Data Head ---
   YRDOY   srad   tmax   tmin  rain   rhum     wind   dewp        par  \
0  24001  16.74  27.64  22.04  0.23  77.47  231.552  20.08  16.353226   
1  24002  14.15  27.54  21.47  0.04  77.42  253.152  19.99  16.353226   
2  24003  12.99  27.71  22.40  1.01  81.63  273.024  21.16  16.353226   
3  24004   9.41  28.04  22.80  2.55  80.49  254.880  21.41  16.353226   
4  24005  13.76  27.96  23.35  1.07  80.22  285.984  21.32  16.353226   

       vapr    dco2      ozon7  evap  
0  2.349886  422.23  84.993684  2.23  
1  2.336834  422.37  76.241667  2.20  


In [ ]:
# import requests
# import pandas as pd
# import numpy as np # Added for calculations
# import os
# from datetime import datetime, timezone, timedelta
# from collections import defaultdict
# import concurrent.futures

# # ==============================================================================
# # DATA FETCHING HELPERS (CO2 and O3)
# # These functions remain largely unchanged but will be called by fetch_weather.
# # ==============================================================================

# def get_co2(start_date_str: str, end_date_str: str, filename: str = 'co2_daily_mlo.txt') -> pd.DataFrame:
#     """Optimized: Fetches daily CO2 data, reading the local file only once."""
#     DATA_URL = 'https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_daily_mlo.txt'
#     req_start_date = pd.to_datetime(start_date_str)
#     req_end_date = pd.to_datetime(end_date_str)
    
#     download_needed = True
#     df = pd.DataFrame()

#     if os.path.exists(filename):
#         print(f"📄 Local file '{filename}' found. Checking date range...")
#         try:
#             df = _read_noaa_file(filename)
#             local_min_date = df['date'].min()
#             local_max_date = df['date'].max()
#             if req_start_date >= local_min_date and req_end_date <= local_max_date:
#                 print("   - Local data is sufficient.")
#                 download_needed = False
#             else:
#                 print("   - Local data is outdated. Download required.")
#         except (pd.errors.EmptyDataError, KeyError, IndexError):
#             print("   - Local file corrupt. Re-downloading.")

#     if download_needed:
#         try:
#             print(f"⬇️ Downloading CO2 data from NOAA...")
#             response = requests.get(DATA_URL, timeout=15)
#             response.raise_for_status()
#             with open(filename, 'w', encoding='utf-8') as file:
#                 file.write(response.text)
#             print(f"✅ CO2 Download complete.")
#             df = _read_noaa_file(filename)
#         except requests.exceptions.RequestException as e:
#             print(f"❌ Error fetching CO2 URL: {e}")
#             return pd.DataFrame()

#     if not df.empty:
#         mask = (df['date'] >= req_start_date) & (df['date'] <= req_end_date)
#         print("✅ CO2 processing complete.")
#         return df.loc[mask, ['date', 'ppm']].reset_index(drop=True)
    
#     return pd.DataFrame()

# def _read_noaa_file(filepath: str) -> pd.DataFrame:
#     """Helper function to read and parse the NOAA data file."""
#     df = pd.read_csv(filepath, comment="#", sep=r'\s+', names=["yr", "mon", "day", "decimal", "ppm"])
#     df['date'] = pd.to_datetime(df[['yr', 'mon', 'day']].rename(columns={'yr': 'year', 'mon': 'month', 'day': 'day'}))
#     return df

# def get_o3(api_key: str, lat: float, lon: float, start_str: str, end_str: str) -> pd.DataFrame:
#     """Optimized: Fetches O3 data and returns a clean DataFrame."""
#     if not api_key or api_key == "YOUR_API_KEY_HERE":
#         print("🛑 O3 Fetching Skipped: Invalid OpenWeatherMap API key.")
#         return pd.DataFrame()

#     try:
#         start_dt = datetime.strptime(start_str, "%Y-%m-%d")
#         end_dt = datetime.strptime(end_str, "%Y-%m-%d").replace(hour=23, minute=59, second=59)
#         start_ts = int(start_dt.replace(tzinfo=timezone.utc).timestamp())
#         end_ts = int(end_dt.replace(tzinfo=timezone.utc).timestamp())

#         api_url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_ts}&end={end_ts}&appid={api_key}"
        
#         print(f"⬇️ Fetching O3 data from OpenWeatherMap API...")
#         response = requests.get(api_url)
#         response.raise_for_status()
#         data = response.json()

#         if 'list' not in data or not data['list']:
#             print("No O3 data found for the specified location and date range.")
#             return pd.DataFrame()
        
#         daily_data = defaultdict(list)
#         for entry in data['list']:
#             local_date = datetime.fromtimestamp(entry['dt'], tz=timezone(timedelta(hours=5, minutes=30))).date()
#             daily_data[local_date].append(entry['components'].get('o3', 0))

#         records = [{'date': date, 'OZON7': sum(vals) / len(vals)} for date, vals in sorted(daily_data.items())]
        
#         if not records:
#             return pd.DataFrame()
        
#         print("✅ O3 processing complete.")
#         return pd.DataFrame(records)

#     except requests.exceptions.RequestException as e:
#         print(f"❌ Network error fetching O3 data: {e}")
#     except (KeyError, IndexError, ZeroDivisionError) as e:
#         print(f"❌ Error processing O3 data: {e}.")
    
#     return pd.DataFrame()

# # ==============================================================================
# # MAIN ORCHESTRATOR FUNCTION
# # This function now handles everything: fetching all data, merging, and writing.
# # ==============================================================================

# def create_weather_file(api_key, lat, lon, start_date, end_date, site_name="SITE", insi="XXXX", elev=None):
#     """
#     Fetches weather, CO2, and O3 data concurrently, merges them,
#     and writes a DSSAT-compatible .WTH file.
#     """
#     # --- 1. CONCURRENTLY FETCH ALL DATA SOURCES ---
#     weather_df, co2_df, o3_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
#     print("--- Starting Concurrent Data Fetch ---")
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         future_weather = executor.submit(_fetch_nasa_weather, lat, lon, start_date, end_date)
#         future_co2 = executor.submit(get_co2, start_date, end_date)
#         future_o3 = executor.submit(get_o3, api_key, lat, lon, start_date, end_date)
        
#         weather_df = future_weather.result()
#         co2_df = future_co2.result()
#         o3_df = future_o3.result()
#     print("--- Concurrent Data Fetch Complete ---\n")

#     if weather_df.empty:
#         print("❌ Critical Error: Could not fetch base weather data from NASA. Aborting.")
#         return

#     # --- 2. MERGE AND PROCESS DATA ---
#     # Start with the base weather data
#     df = weather_df
    
#     # Merge CO2 data if available
#     if not co2_df.empty:
#         df = pd.merge(df, co2_df, on='date', how='left')
#         df = df.rename(columns={"ppm": "dco2"})
    
#     # Merge O3 data if available
#     if not o3_df.empty:
#         # Ensure date types match before merging
#         o3_df['date'] = pd.to_datetime(o3_df['date'])
#         df = pd.merge(df, o3_df, on='date', how='left')
#         df = df.rename(columns={"OZON7": "ozon7"})

#     # Use fillna() for robust handling of missing data
#     df['dco2'] = df.get('dco2', 421.0).fillna(421.0)
#     df['ozon7'] = df.get('ozon7', 50).fillna(50)

#     # --- 3. PERFORM CALCULATIONS ---
#     df["wind"] = df["wind"] * 86.4  # m/s → km/day
#     df["month"] = df["date"].dt.month
#     monthly_srad_avg = df.groupby("month")["srad"].mean().to_dict()
#     df["par"] = df["month"].map(monthly_srad_avg)
#     df["vapr"] = 0.6108 * np.exp((17.27 * df["dewp"]) / (df["dewp"] + 237.3))
#     df["YRDOY"] = df["date"].dt.strftime("%y%j")
    
#     tav = ((df["tmax"] + df["tmin"]) / 2).mean()
#     amp = (df["tmax"].mean() - df["tmin"].mean())

#     # --- 4. PREPARE AND WRITE THE .WTH FILE ---
#     df_out = df[["YRDOY","srad","tmax","tmin","rain","rhum","wind",
#                  "dewp","par","vapr","dco2","ozon7","evap"]]

#     output_dir = "weather"
#     os.makedirs(output_dir, exist_ok=True)
#     start_yrdoy = df_out["YRDOY"].iloc[0]
#     wth_file = os.path.join(output_dir, f"{insi}_{start_yrdoy}.WTH")

#     with open(wth_file, "w") as f:
#         f.write(f"*WEATHER DATA : {site_name},{'NASA'}\n\n")
#         f.write("@ INSI      LAT     LONG   ELEV   TAV   AMP REFHT WNDHT   CCO2\n")
#         f.write(f"  {insi:<4s}  {lat:7.3f} {lon:7.3f} {elev if elev else 100:5.1f}  {tav:5.1f}  {amp:5.1f}   2.0   2.0  {df['dco2'].mean():6.1f}\n\n")
#         f.write("@  DATE  SRAD  TMAX  TMIN  RAIN  RHUM  WIND  DEWP   PAR  VAPR   DCO2  OZON7  EVAP\n")
#         for _, row in df_out.iterrows():
#             f.write(f"{int(row.YRDOY):7d} {row.srad:5.1f} {row.tmax:5.1f} {row.tmin:5.1f} "
#                     f"{row.rain:5.1f} {row.rhum:5.1f} {row.wind:5.1f} {row.dewp:5.1f} "
#                     f"{row.par:6.1f} {row.vapr:5.2f} {row.dco2:6.1f} {int(row.ozon7):6d} {row.evap:5.1f}\n")

#     print(f"✅ Weather file successfully saved at: {wth_file}")
#     return df_out

# def _fetch_nasa_weather(lat, lon, start, end):
#     """Helper function dedicated to fetching NASA POWER data."""
#     print("⬇️ Fetching base weather data from NASA POWER...")
#     try:
#         url = "https://power.larc.nasa.gov/api/temporal/daily/point"
#         params = {
#             "start": pd.to_datetime(start).strftime("%Y%m%d"),
#             "end": pd.to_datetime(end).strftime("%Y%m%d"),
#             "latitude": lat, "longitude": lon, "community": "AG",
#             "parameters": "ALLSKY_SFC_SW_DWN,T2M_MAX,T2M_MIN,T2MDEW,PRECTOTCORR,WS2M,RH2M,EVPTRNS",
#             "format": "JSON"
#         }
#         response = requests.get(url, params=params).json()
#         data = response["properties"]["parameter"]
        
#         df = pd.DataFrame(data).reset_index().rename(columns={"index": "date"})
#         df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
        
#         df = df.rename(columns={
#             "ALLSKY_SFC_SW_DWN":"srad", "T2M_MAX":"tmax", "T2M_MIN":"tmin", "T2MDEW":"dewp",
#             "PRECTOTCORR":"rain", "WS2M":"wind", "RH2M":"rhum", "EVPTRNS":"evap"
#         })
#         print("✅ NASA Weather processing complete.")
#         return df
#     except Exception as e:
#         print(f"❌ Error fetching NASA weather data: {e}")
#         return pd.DataFrame()

# # ==============================================================================
# # SCRIPT EXECUTION
# # ==============================================================================
# if __name__ == "__main__":
#     # --- Configuration ---
#     API_KEY = "eddbc7c0c9e63e225934add809624c6e" # Your OpenWeatherMap API Key
#     LATITUDE = 13.08
#     LONGITUDE = 80.27
#     START_DATE = "2025-01-01"
#     END_DATE = "2025-03-31"
    
#     # --- Execute the Main Function ---
#     final_data = create_weather_file(
#         api_key=API_KEY,
#         lat=LATITUDE,
#         lon=LONGITUDE,
#         start_date=START_DATE,
#         end_date=END_DATE,
#         site_name="CHENNAI",
#         insi="IN01",
#         elev=6
#     )

#     if final_data is not None:
#         print("\n--- Final Data Head ---")
#         print(final_data.head())
